In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC



In [2]:
# Upload the data and clean up.
data = pd.read_csv("h1b_data.csv")
data = data.dropna(axis=0, how='any')

data = data[~data.CASE_STATUS.str.contains('WITHDRAWN')]
data = data[~data.CASE_STATUS.str.contains('WITHDRAWN')]
data = data[~data.CASE_STATUS.str.contains('ASSIGNED')]
data = data[~data.CASE_STATUS.str.contains('INVALIDATED')]


data = data.replace(to_replace="CERTIFIED", value=1)
data = data.replace(to_replace="DENIED", value=0)
data = data.replace(to_replace="REJECTED", value=0)
data = data.drop('lon', 1)
data = data.drop('lat', 1)
data = data.drop('JOB_TITLE', 1)
data = data.drop('Unnamed: 0', 1)

data['FULL_TIME_POSITION'].replace(['Y','N'],['1','0'],inplace=True)
Cities,States = data['WORKSITE'].str.split(', ',1).str
city_to_state = dict(zip(data['WORKSITE'], States))
data['WORKSITE']= data['WORKSITE'].map(city_to_state)
data = data[~data.WORKSITE.str.match('NA')]
data.WORKSITE = pd.Categorical(data.WORKSITE)
data['WORKSITE'] = data.WORKSITE.cat.codes

#Create a feature for number of applications per company.
lookup = data.EMPLOYER_NAME.value_counts()
data['APPS_PER_COMPANY']= data['EMPLOYER_NAME'].map(lookup)

#Get rid of the companies with less than 4 applications.
data=data[data["APPS_PER_COMPANY"]>3]

# #Create a feature for the success rate per company.
# lookup = data.EMPLOYER_NAME.value_counts()
# lookup2 = dict(zip(data.EMPLOYER_NAME.unique(),np.zeros((data.EMPLOYER_NAME.value_counts().size))))
# lkp=lookup.size
# for i in range(0,lkp):
#     lookup2[lookup.index[i]]=(np.sum(data[data.EMPLOYER_NAME==lookup.index[i]].CASE_STATUS.values))/lookup[i]
# data['SUCCESS_RATE']= data['APPS_PER_COMPANY'].map(lookup2)

#Clean up SOC_NAME metadata.
data['SOC_NAME'] = data['SOC_NAME'].str.upper()
data['SOC_NAME'] = data['SOC_NAME'].map(lambda x: str(x).lstrip('*').rstrip('*'))
data.SOC_NAME = data.SOC_NAME.str.replace('[^a-zA-Z]', '')
data.SOC_NAME = data.SOC_NAME.str.replace('ALLOTHER', '')

#Get rid of SOC's that have less than 4 appearances in data.
#Create a feature for number of applications per SOC type.
lookup = data.SOC_NAME.value_counts()
data['APPS_PER_SOC']= data['SOC_NAME'].map(lookup)
data=data[data["APPS_PER_SOC"]>3]

#Convert SOC and EMPLOYER_NAME to numeric values.
data.SOC_NAME = pd.Categorical(data.SOC_NAME)
data['SOC_NAME'] = data.SOC_NAME.cat.codes
data.EMPLOYER_NAME = pd.Categorical(data.EMPLOYER_NAME)
data['EMPLOYER_NAME'] = data.EMPLOYER_NAME.cat.codes


,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,APPS_PER_COMPANY,APPS_PER_SOC
19,1,28909,97,1,241842.0,2016.0,47,6,1658
27,1,20408,97,1,215862.0,2016.0,4,6,1658
31,1,46096,97,1,226699.0,2016.0,13,13,1658
33,1,50200,97,1,159370.0,2016.0,9,6,1658
38,1,13843,97,1,130853.0,2016.0,13,16,1658
39,0,34586,97,1,187200.0,2016.0,48,8,1658
40,1,8447,97,1,89107.0,2016.0,22,5,1658
41,1,41859,97,1,130853.0,2016.0,13,24,1658
42,1,42409,97,1,102190.0,2016.0,10,168,1658
43,1,1066,97,1,197683.0,2016.0,4,7,1658


In [2]:
# data.to_csv('h1b_semiprocessed.csv')
data = pd.read_csv("h1b_semiprocessed.csv")


In [3]:
#Split the data into training and test set.
labels = data.CASE_STATUS
data = data.drop('CASE_STATUS', 1)
data = data.drop('Unnamed: 0', 1)
data = data.drop('Unnamed: 0.1', 1)
list(data.columns.values)


['EMPLOYER_NAME',
 'SOC_NAME',
 'FULL_TIME_POSITION',
 'PREVAILING_WAGE',
 'YEAR',
 'WORKSITE',
 'APPS_PER_COMPANY',
 'APPS_PER_SOC']

In [5]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
# from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.01)
clf = SGDClassifier(loss="hinge",shuffle=True, penalty="l2")
clf.fit(X_train, y_train)
# # SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
# #        eta0=0.0, fit_intercept=True, l1_ratio=0.15,
# #        learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
# #        n_jobs=1, penalty='elasticnet', power_t=0.5, random_state=None,
# #        shuffle=True, tol=None, verbose=0, warm_start=False)
clf.score(X_test, y_test)
# # clf = SVC()
# # clf.fit(X_train, y_train) 
# # y_pred = clf.predict(np.random.random((2,5)))
# # clf.accuracy_score(y_test, y_pred)


/Users/belizgunel/anaconda/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.97494789672919058